In [65]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Data Crawling-----------------------------------------------------------------------------------------
url = 'https://www.booking.com/searchresults.zh-tw.html?label=gen173nr-1FCAQoggJCDXNlYXJjaF_lj7DljJdIMFgEaOcBiAEBmAEwuAEXyAEM2AEB6AEB-AEGiAIBqAIDuALywuOqBsACAdICJGZkYmNjYWNlLWE3MDktNGE2Yy1hNjIyLWQxN2FlNTNjMzM0MdgCBeACAQ&aid=3976&ss=%E5%8F%B0%E5%8C%97&checkin=2023-11-26&lang=zh-tw&sb=1&src_elem=sb&checkout=2023-11-29&offset=0'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}
hotels_data = []
i = 0
while(len(hotels_data) < 100):
    #print(len(hotels_data))
    #print(url)
    url = url[:url.rfind("=")+1]
    url += str(i*25)
    response  = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    hotels = soup.findAll('div', {'data-testid': 'property-card'})

    for hotel in hotels:
        name_element = hotel.select_one('[data-testid="title"]')
        name = name_element.text.strip()

        location_element = hotel.select_one('[data-testid="address"]')
        location = location_element.text.strip()

        distance_element = hotel.select_one('[data-testid="distance"]')
        distance = distance_element.text.strip()

        price_element = hotel.select_one('[data-testid="price-and-discounted-price"]')
        price = price_element.text.strip()

        rating_element = hotel.select_one('[data-testid="review-score"]')
        rating = rating_element.text.strip()

        comment_element = hotel.find('div', {'class': 'a3b8729ab1 e6208ee469 cb2cbb3ccb'})
        comment = comment_element.text.strip()

        hotels_data.append([
            name,
            location,
            price,
            rating,
            distance,
            comment
        ])

    hotels_data = pd.DataFrame(hotels_data)
    hotels_data.drop_duplicates(inplace=True)
    hotels_data = hotels_data.values.tolist()
    #print(hotels_data)
    i += 1

#csv for comparison
hotels = pd.DataFrame(hotels_data)
hotels.head()
hotels.to_csv('raw.csv', header=True, index=False)

#Data cleaning-----------------------------------------------------------------------------------------
for row in hotels_data:
    #Convert type of price
    row[2] = float("".join(row[2][4:].split(",")))
        
    #Convert type of rating and handle missing values.
    if row[3] == '':
        row[3] = -1.0
    else:
        row[3] = row[3][0:3]
        row[3] = float(row[3])
    if row[5] == '': row[5] = "None"
        
    #convert type of distance
    if row[4][-2:] == '公里':
        row[4] = int(1000 * float(row[4].split(" ")[1]))
    else:
        row[4] = int(row[4].split(" ")[1])
        
#remove outliers
#hotels_data = [p for p in hotels_data if p[2] < 10000]
    
#csv for comparison
cooked = hotels = pd.DataFrame(hotels_data)
cooked.head()
cooked.to_csv('cooked.csv', header=True, index=False)